In [41]:
import fastai.test_utils

In [42]:
fastai.test_utils.show_install()



```text
=== Software === 
python       : 3.8.8
fastai       : 2.3.0
fastcore     : 1.3.19
fastprogress : 0.2.7
torch        : 1.7.1
torch cuda   : 10.2 / is **Not available** 

=== Hardware === 
No GPUs available 

=== Environment === 
platform     : Linux-5.10.16.3-microsoft-standard-WSL2-x86_64-with-glibc2.10
distro       : #1 SMP Fri Apr 2 22:23:49 UTC 2021
conda env    : fastbook
python       : /home/sandmann/anaconda3/envs/fastbook/bin/python
sys.path     : /home/sandmann/repo/fastai/dev_nbs/course/selfmade
/home/sandmann/anaconda3/envs/fastbook/lib/python38.zip
/home/sandmann/anaconda3/envs/fastbook/lib/python3.8
/home/sandmann/anaconda3/envs/fastbook/lib/python3.8/lib-dynload

/home/sandmann/.local/lib/python3.8/site-packages
/home/sandmann/anaconda3/envs/fastbook/lib/python3.8/site-packages
/home/sandmann/anaconda3/envs/fastbook/lib/python3.8/site-packages/IPython/extensions
/home/sandmann/.ipython
no supported gpus found on this system
```

Please make sure to include openin

# Human Numbers

In [1]:
from fastai.text.all import *

In [2]:
bs = 64

# Data

In [3]:
path = untar_data(URLs.HUMAN_NUMBERS)

In [4]:
path.ls()

(#2) [Path('/home/sandmann/.fastai/data/human_numbers/valid.txt'),Path('/home/sandmann/.fastai/data/human_numbers/train.txt')]

In [5]:
def readnums(fn): return ', '.join([o.strip() for o in (path/fn).readlines()])

In [6]:
train_txt = readnums('train.txt')

In [7]:
valid_txt = readnums('valid.txt')

In [8]:
train_txt[:80]

'one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirt'

In [9]:
valid_txt[:80]

'eight thousand one, eight thousand two, eight thousand three, eight thousand fou'

In [10]:
train_tok = tokenize1(train_txt, tok=WordTokenizer())
valid_tok = tokenize1(valid_txt, tok=WordTokenizer())

In [11]:
dsets = Datasets([train_tok, valid_tok], tfms=Numericalize, dl_type=LMDataLoader, splits=[[0], [1]])

In [12]:
dls = dsets.dataloaders(bs=bs, val_bs=bs)

In [13]:
dsets.show((dsets.train[0][0][:80],)) # ???

In [14]:
len(dsets.valid[0][0])

13017

In [15]:
dls.seq_len, len(dls.valid)

(72, 3)

In [16]:
13017/72/bs # -> len(dls.valid)

2.8248697916666665

In [17]:
dls.valid

In [18]:
it = iter(dls.valid)
x1, y1 = next(it)
x2, y2 = next(it)
x3, y3 = next(it)
it.close()

In [19]:
x1.numel() + x2.numel() + x3.numel()

12992

In [20]:
x1.shape, x2.shape, x3.shape

(torch.Size([64, 72]), torch.Size([64, 72]), torch.Size([64, 59]))

In [21]:
x1.shape, y1.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [22]:
x3.shape, y3.shape

(torch.Size([64, 59]), torch.Size([64, 59]))

In [23]:
x1[0]

LMTensorText([ 2, 19, 11, 12,  9, 19, 11, 13,  9, 19, 11, 14,  9, 19, 11, 15,  9, 19,
        11, 16,  9, 19, 11, 17,  9, 19, 11, 18,  9, 19, 11, 19,  9, 19, 11, 20,
         9, 19, 11, 29,  9, 19, 11, 30,  9, 19, 11, 31,  9, 19, 11, 32,  9, 19,
        11, 33,  9, 19, 11, 34,  9, 19, 11, 35,  9, 19, 11, 36,  9, 19, 11, 37])

In [24]:
y1[0]

TensorText([19, 11, 12,  9, 19, 11, 13,  9, 19, 11, 14,  9, 19, 11, 15,  9, 19, 11,
        16,  9, 19, 11, 17,  9, 19, 11, 18,  9, 19, 11, 19,  9, 19, 11, 20,  9,
        19, 11, 29,  9, 19, 11, 30,  9, 19, 11, 31,  9, 19, 11, 32,  9, 19, 11,
        33,  9, 19, 11, 34,  9, 19, 11, 35,  9, 19, 11, 36,  9, 19, 11, 37,  9])

In [25]:
v = dls.vocab

In [26]:
type(v)

list

In [27]:
def translate(t): return ' '.join(v[n] for n in t)

In [28]:
translate(x1[0])

'xxbos eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand eighteen'

In [29]:
translate(y1[0])

'eight thousand one , eight thousand two , eight thousand three , eight thousand four , eight thousand five , eight thousand six , eight thousand seven , eight thousand eight , eight thousand nine , eight thousand ten , eight thousand eleven , eight thousand twelve , eight thousand thirteen , eight thousand fourteen , eight thousand fifteen , eight thousand sixteen , eight thousand seventeen , eight thousand eighteen ,'

In [30]:
translate(x2[0])

', eight thousand nineteen , eight thousand twenty , eight thousand twenty one , eight thousand twenty two , eight thousand twenty three , eight thousand twenty four , eight thousand twenty five , eight thousand twenty six , eight thousand twenty seven , eight thousand twenty eight , eight thousand twenty nine , eight thousand thirty , eight thousand thirty one , eight thousand thirty two , eight thousand thirty three'

In [31]:
translate(x3[0])

', eight thousand thirty four , eight thousand thirty five , eight thousand thirty six , eight thousand thirty seven , eight thousand thirty eight , eight thousand thirty nine , eight thousand forty , eight thousand forty one , eight thousand forty two , eight thousand forty three , eight thousand forty four , eight thousand forty five'

In [32]:
translate(x1[1])

', eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand'

# Models

## Single fully connected model

In [33]:
dls = dsets.dataloaders(bs=bs, seq_len=3)

In [34]:
x,y = dls.one_batch()
x.shape, y.shape

(torch.Size([64, 3]), torch.Size([64, 3]))

In [35]:
nv = len(v)
nv

40

In [36]:
nh = 64

In [37]:
def loss4(input, target): return F.cross_entropy(input, target[:,-1])
def acc4(input, target): return accuracy(input, target[:,-1])

In [38]:
class Model0(Module):
    def __init__(self):
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = self.bn(F.relu(self.h_h(self.i_h(x[:,0]))))
        if x.shape[1] > 1:
            h += self.i_h(x[:,1])
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[1] > 2:
            h += self.i_h(x[:,2])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [39]:
learn = Learner(dls, Model0(), loss_func=loss4, metrics=acc4)

In [40]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.470132,3.389263,0.139476,00:08
1,2.522893,2.497188,0.451976,00:07
2,2.045766,2.163740,0.459329,00:07
3,1.861677,2.065527,0.463465,00:09
4,1.795365,2.039244,0.463465,00:08
5,1.781597,2.035885,0.467371,00:08


## Loop

In [41]:
class Model1(Module):
    def __init__(self):
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh, device=x.device)
        for i in range(x.shape[1]):
            h += self.i_h(x[:,i])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [42]:
learn = Learner(dls, Model1(), loss_func=loss4, metrics=acc4)

In [43]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4,time
0,3.498695,3.502387,0.096278,00:09
1,2.585205,2.653282,0.414522,00:09
2,2.065539,2.268891,0.463465,00:07
3,1.866867,2.137375,0.466912,00:09
4,1.797158,2.096108,0.466682,00:09
5,1.782675,2.090410,0.466452,00:09


## Multi fully connected model

In [44]:
dls = dsets.dataloaders(bs=bs, seq_len=20)

In [45]:
x, y = dls.one_batch()
x.shape, y.shape

(torch.Size([64, 20]), torch.Size([64, 20]))

In [46]:
class Model2(Module):
    def __init__(self):
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        res = []
        for i in range(x.shape[1]):
            #h += self.i_h(x[:,i]) # += doesnt work for some reason
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.h_o(self.bn(h)))
        return torch.stack(res, dim=1)
        

In [47]:
learn = Learner(dls, Model2(), loss_func=CrossEntropyLossFlat(), metrics=accuracy)

In [48]:
learn.fit_one_cycle(10, 1e-4)

epoch,train_loss,valid_loss,accuracy,time
0,3.843050,3.949107,0.019366,00:03
1,3.756002,3.803862,0.022656,00:03
2,3.573745,3.587763,0.053575,00:03
3,3.349103,3.386632,0.110014,00:04
4,3.133504,3.224458,0.185606,00:03
5,2.952198,3.106509,0.225663,00:03
6,2.813930,3.030595,0.247088,00:03
7,2.718174,2.989381,0.256084,00:05
8,2.659418,2.973421,0.259044,00:03
9,2.628728,2.971048,0.259186,00:02


# Maintain State

In [38]:
class Model3(Module):
    def __init__(self):
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(bs,nh).cuda()
        
    def forward(self,x):
        if self.h.shape[0] != x.shape[0]: self.h = torch.zeros(x.shape[0], nh).cuda()
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = self.bn(F.relu(self.h_h(h)))
            res.append(h)
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res
    
    def reset(self): self.h = torch.zeros(bs,nh).cuda()

In [ ]:
learn = Learner(dls, Model3(), loss_func=CrossEntropyLossFlat, metrics=accuracy)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(20, 3e-3)

# nn.RNN

In [40]:
class Model4(Module):
    def __init__(self):
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.RNN(nh,nh, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self,x):
        if x.shape[0] != self.h.shape[1]: self.h = torch.zeros(1,x.shape[0],nh).cuda()
        res, h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [ ]:
learn = Learner(dls, Model4(), loss_func=CrossEntropyLossFlat(), metrics=accuracy)

In [ ]:
learn.fit_one_cycle(20, 3e-3)